# EFL Data Colloector
need to run EFL.py first

In [23]:
%run 'EFL.py'

## Training Engine
Machines are trained here.

In [894]:
%run 'EFL.py'
machine = Machine(
    IsingModel(SSLattice(width=32,depth=100,pattern=[-1,-1,1])),
    FreeFermion(size=64,mass=0.,c=4.0),method='weighted')
try:
    machine.load()
except:
    print('File not found! Train new machine.')

INFO:tensorflow:Restoring parameters from ./machine/Ising(SSLatt(32,100,UUI))FF(64,0.50,4.0)weighted
File not found! Train new machine.


In [901]:
for i in range(5):
    machine.train(16, learning_rate=0.005, beta1=0.9, beta2=0.9)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))

(8.2414611868045543e-05, array([ 0.053,  0.005,  0.000,  0.000,  0.000]))
(0.000137758269367625, array([ 0.054,  0.005,  0.000,  0.000,  0.000]))
(8.7041832869568908e-05, array([ 0.054,  0.005,  0.000,  0.000,  0.000]))
(3.3677494530541446e-05, array([ 0.053,  0.000,  0.000,  0.000,  0.000]))
(0.00010222809758183779, array([ 0.053,  0.000,  0.000,  0.000,  0.000]))


I/O: save machine

In [902]:
machine.save()

INFO:tensorflow:Saving parameters to ./machine/Ising(SSLatt(32,100,UUI))FF(64,0.50,4.0)weighted


----

In [400]:
machine.session.run(machine.model.J.assign(np.array([0.26,0.05,0.05,0.05,0.05])))

array([ 0.260,  0.050,  0.050,  0.050,  0.050])

In [537]:
machine.session.run(machine.model.h)

11.090354888959125

## J Sequence
Given trained machine, run additional trainings to get the time sequence of J.
### Collect Jseq
Data format: `Jseq = {machine: [[J1,J2,...], ...], ...}`

In [903]:
try:
    Jseq = load('Jseq')
except FileNotFoundError:
    Jseq = {}
try:
    Js = Jseq[machine.info]
except KeyError:
    Js = []
print('len: ', len(Js))

len:  0


In [904]:
for i in range(20):
    machine.train(16, learning_rate=0.005, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Js.append(list(machine.session.run(machine.model.J)))
print('len: ', len(Js))
print('avg: ', np.mean(Js,axis=0))
print('std: ', np.std(Js,axis=0))

(6.1154497289490542e-05, array([ 0.053,  0.000,  0.000,  0.000,  0.000]))
(4.7208842235455734e-05, array([ 0.054,  0.000,  0.000,  0.000,  0.000]))
(0.00019264132659747401, array([ 0.053,  0.001,  0.000,  0.000,  0.000]))
(5.1873798672456684e-05, array([ 0.053,  0.003,  0.000,  0.000,  0.000]))
(8.1848297625627966e-05, array([ 0.053,  0.003,  0.000,  0.000,  0.000]))
(4.608158186283059e-05, array([ 0.054,  0.002,  0.000,  0.000,  0.000]))
(0.00013874977607373988, array([ 0.054,  0.002,  0.000,  0.000,  0.000]))
(1.4337901427456259e-05, array([ 0.054,  0.003,  0.000,  0.000,  0.000]))
(0.00024004388402431399, array([ 0.054,  0.003,  0.000,  0.000,  0.000]))
(2.9431663065135773e-05, array([ 0.054,  0.002,  0.000,  0.000,  0.000]))
(6.7678291814893428e-05, array([ 0.054,  0.001,  0.000,  0.000,  0.000]))
(0.00010957012407684478, array([ 0.053,  0.002,  0.000,  0.000,  0.000]))
(7.2164600028881313e-05, array([ 0.053,  0.006,  0.001,  0.000,  0.000]))
(0.00011145504125363073, array([ 0.054,

In [905]:
Jseq[machine.info] = Js
save('Jseq', Jseq)

----

In [758]:
Js=[]

In [906]:
sorted([(mach, np.mean(Js,axis=0)) for mach, Js in Jseq.items()])

[('Ising(SSLatt(16,100,UUI))FF(32,0.00,8.0)random',
  array([ 0.663,  0.201,  0.201,  0.200])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,0.5)random',
  array([ 0.331,  0.002,  0.000,  0.000,  0.000])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,1.0)random',
  array([ 0.272,  0.000,  0.000,  0.000,  0.000])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,1.5)random',
  array([ 0.253,  0.099,  0.099,  0.099,  0.099])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,12.0)random',
  array([ 1.050,  0.238,  0.238,  0.239,  0.239])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,16.0)random',
  array([ 1.404,  0.245,  0.245,  0.245,  0.246])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,2.0)multiple',
  array([ 0.248,  0.196,  0.196,  0.197,  0.198])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,2.0)random',
  array([ 0.258,  0.157,  0.157,  0.157,  0.158])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,2.0)weighted',
  array([ 0.245,  0.197,  0.197,  0.198,  0.199])),
 ('Ising(SSLatt(32,100,UUI))FF(64,0.00,3.0)random',
  array([ 0.31

## Entanglement Feature
Given trained machine, collect entropy predictions
### Collect EFcon
Entanglement feature over consecutive regions.
Data format: `EFcon = {machine: {'Ssys': [S1,...], 'Smdls': {[[S1,...], ...]}}, ...}`

In [729]:
try:
    EFcon = load('EFcon')
except FileNotFoundError:
    EFcon = {}
try:
    EF = EFcon[machine.info]
    Ssys = EF['Ssys']
    Smdls = EF['Smdls']
except KeyError:
    Ssys = list(machine.data_server.fetch('consecutive')[machine.model.Ssys])
    Smdls = []
    EF = {'Ssys': Ssys, 'Smdls': Smdls}
print('len: ', len(Smdls))

len:  0


In [732]:
for i in range(5):
    machine.train(16, learning_rate=0.005, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    Smdl = list(machine.session.run(machine.model.Smdl, 
                            machine.data_server.fetch('consecutive')))
    Smdls.append(Smdl)
print('len: ', len(Smdls))
print('error: ', np.mean([np.sqrt(np.mean(np.square(np.array(Smdl)/np.array(Ssys)-1))) for Smdl in Smdls]))

(0.00094864522821205193, array([ 0.708,  0.233,  0.233,  0.234,  0.234]))
(0.00055716851500628754, array([ 0.707,  0.231,  0.231,  0.232,  0.232]))
(0.00049439989314737352, array([ 0.701,  0.225,  0.225,  0.225,  0.225]))
(0.0010689886132386555, array([ 0.705,  0.227,  0.227,  0.227,  0.228]))
(0.00078033268832330971, array([ 0.708,  0.226,  0.226,  0.226,  0.227]))
len:  15
error:  0.134904315085


In [733]:
EFcon[machine.info] = {'Ssys': Ssys, 'Smdls': Smdls}
save('EFcon', EFcon)

### Collect EFmul
Entanglement feature over multiregions.
Data format: `EFmul = {machine: {n: [[Ssys, Smdl], ...], ...},...}`

In [734]:
try:
    EFmul = load('EFmul')
except FileNotFoundError:
    EFmul = {}
try:
    ndict = EFmul[machine.info]
except KeyError:
    ndict = {n:[] for n in range(1, machine.model.partitions//2+1)}
print('len: ', len(ndict[1]))

len:  0


In [736]:
for i in range(5):
    machine.train(16, check=16)
    print(machine.session.run((machine.model.MSE, machine.model.J), machine.feed))
    for n, pairs in ndict.items():
        pairs += [[Ssys, Smdl] for Ssys, Smdl in zip(
                    *machine.session.run([
                        machine.model.Ssys, machine.model.Smdl], 
                        machine.data_server.fetch(('multiple',n))))]
print('len: ', len(ndict[1]))
print({n:float('{0:.3f}'.format(np.sqrt(np.mean([(a/b-1.)**2 for [a, b] in pairs])))) for n, pairs in ndict.items()})

(0.0010616919614045198, array([ 0.700,  0.221,  0.221,  0.221,  0.221]))
(0.00071267595131047252, array([ 0.711,  0.236,  0.236,  0.237,  0.238]))
(0.001527546720059988, array([ 0.701,  0.234,  0.234,  0.235,  0.236]))
(0.0027654592551600516, array([ 0.692,  0.227,  0.227,  0.228,  0.228]))
(0.00065503921225155818, array([ 0.703,  0.243,  0.243,  0.243,  0.244]))
len:  70
{1: 0.167, 2: 0.12, 3: 0.073, 4: 0.058, 5: 0.042, 6: 0.036, 7: 0.032, 8: 0.024, 9: 0.03, 10: 0.029, 11: 0.026, 12: 0.024, 13: 0.019, 14: 0.015, 15: 0.02, 16: 0.037}


In [737]:
EFmul[machine.info] = ndict
save('EFmul', EFmul)

In [739]:
for mach, ndict in EFmul.items():
    print(mach,{n:float('{0:.3f}'.format(np.sqrt(np.mean([(a/b-1.)**2 for [a, b] in pairs])))) for n, pairs in ndict.items()})

Ising(SSLatt(32,100,UUI))FF(64,0.00,2.0)random {1: 0.12, 2: 0.061, 3: 0.04, 4: 0.025, 5: 0.024, 6: 0.022, 7: 0.027, 8: 0.023, 9: 0.019, 10: 0.015, 11: 0.015, 12: 0.012, 13: 0.016, 14: 0.014, 15: 0.018, 16: 0.023}
Ising(SSLatt(32,100,UUI))FF(64,0.00,8.0)random {1: 0.167, 2: 0.12, 3: 0.073, 4: 0.058, 5: 0.042, 6: 0.036, 7: 0.032, 8: 0.024, 9: 0.03, 10: 0.029, 11: 0.026, 12: 0.024, 13: 0.019, 14: 0.015, 15: 0.02, 16: 0.037}
Ising(SSLatt(32,100,UUI))FF(64,0.00,4.0)random {1: 0.08, 2: 0.06, 3: 0.039, 4: 0.037, 5: 0.028, 6: 0.029, 7: 0.024, 8: 0.027, 9: 0.023, 10: 0.021, 11: 0.019, 12: 0.016, 13: 0.016, 14: 0.016, 15: 0.014, 16: 0.025}
Ising(SSLatt(16,100,UUI))FF(32,0.00,8.0)random {1: 0.075, 2: 0.073, 3: 0.045, 4: 0.038, 5: 0.037, 6: 0.025, 7: 0.026, 8: 0.037}
Ising(SSLatt(32,100,UUI))FF(64,0.00,8.0)multiple {1: 0.053, 2: 0.041, 3: 0.046, 4: 0.051, 5: 0.055, 6: 0.053, 7: 0.059, 8: 0.056, 9: 0.055, 10: 0.055, 11: 0.052, 12: 0.047, 13: 0.041, 14: 0.033, 15: 0.02, 16: 0.023}
Ising(SSLatt(32,10